### script - do not run these cells 

relative imports fail when run as a script so scripts stays above library

In [1]:
from fastai.vision import *
from fastai.distributed import *
from fastai.script import *
from fastai.utils.mem import *

from local.segmentation.dataset import *
from local.segmentation.models import *
from local.segmentation import metrics
from local.segmentation import losses_binary, losses_multilabel
from local.callbacks import *
from local.optimizers import *

To use this log_lamb_rs, please run 'pip install tensorboardx'. Also you must have Tensorboard running to see results


In [ ]:
# Always keeps this in cell index position: 2
from fastai.vision import *
from fastai.distributed import *
from fastai.script import *
from fastai.utils.mem import *

from local.segmentation.dataset import *
from local.segmentation.models import *
from local.segmentation import metrics
from local.segmentation import losses_binary, losses_multilabel
from local.callbacks import *
from local.optimizers import *


# https://stackoverflow.com/questions/8299270/ultimate-answer-to-relative-python-imports
@call_parse
def main(    
    # data
    PATH:Param("Path which have data", str)="",
    IMAGES:Param("images folder path name", str)="images",
    MASKS:Param("mask folder path name", str)="masks",
    CODES:Param("codes.txt with pixel codes", str)="",
    TRAIN:Param("train.txt with training image names", str)="",
    VALID:Param("valid.txt with validation image names", str)=None,
    TEST:Param("test.txt with test image names", str)=None,
    suffix:Param("suffix for label filenames", str)=".png",
    sample_size:Param("", int)=None,
    bs:Param("Batch size", int)=80,
    size:Param("Image size", str)="224",
    imagenet_pretrained:Param("Whether to normalize with inet stats", int)=1,
    
    # model
    modelname:Param("Model name from segmentation.models", str)="resdunet18",
    modelconfig:Param("JSON dictionary of model config", str)="{}",
    
    # metric
    tracking_metric:Param("Which metric to use for tracking and evaluation", str)="dice",
    void_name:Param("Background class name", str)=None,
    
    # train
    loss_function:Param("Loss function for training", str)="cross_entropy",
    opt:Param("Optimizer for training", str)=None,
    max_lr:Param("Learning Rate", float)=3e-3,
    epochs:Param("Number of max epochs to train", int)=10,
    
    # modelexports
    EXPORT_PATH:Param("Where to export trained model", str)=".",
    
    gpu:Param("GPU to run on, can handle multi gpu", str)=None):
    
    """
    For Multi GPU Run: python ../fastai/fastai/launch.py {--gpus=0123} ./training.py {--your args}
    For Single GPU Run: python ./training.py {--your args}
    """
        
    # Setup init
    gpu = setup_distrib(gpu)
    
    # Args
    if not gpu: print(f"Print args here: ")
        
    # data
    PATH = Path(PATH)
    try: VALID = float(VALID)
    except: passzx
    size = eval(size)
    ssdata = SemanticSegmentationData(PATH, IMAGES, MASKS, CODES, TRAIN,
                                      VALID, TEST, sample_size, bs, size, suffix)
    data = ssdata.get_data()
    if imagenet_pretrained: data.normalize(imagenet_stats)
    else: data.normalize()   
    
    # model
    model, split_fn, copy_conf = get_model(modelname, data, eval(modelconfig))
    if not gpu: print(f"Training with model: {modelname} and config: {copy_conf}")
        
    # learn
    learn = Learner(data, model)
    learn = learn.split(split_fn)
    pretrained = copy_conf["pretrained"]
    if pretrained: learn.freeze()
    learn.path, learn.model_dir = Path(EXPORT_PATH), 'models'

    # loss
    try: loss = getattr(losses_binary, loss_function)
    except: loss = getattr(losses_multilabel, loss_function)  
    learn.loss_func = loss 
    if not gpu: print(f"Training with loss: {learn.loss_func}")
        
    # metric
    metric = getattr(metrics, tracking_metric)
    if not gpu: print(f"Training with metric: {metric}")
    if tracking_metric in ["multilabel_dice", "multilabel_iou"]: metric = partial(metric, c=learn.data.c)
    if tracking_metric == "foreground_acc": 
        void_code = np.where(learn.data.classes == void_name)[0].item()
        metric = partial(metric, void_code=void_code)
    learn.metrics = [metric]

    # callbacks
    save_cb = SaveDistributedModelCallback(learn, tracking_metric, "max", name=f"best_of_{modelname}", gpu=gpu)
    csvlog_cb = CSVDistributedLogger(learn, 'training_log', append=True, gpu=gpu)
    nan_cb = TerminateOnNaNCallback()
    cbs = [save_cb, csvlog_cb, nan_cb]
        
    # optimizer / scheduler
    alpha, mom, eps = 0.99, 0.9, 1e-8
    if   opt=='adam':        opt_func = partial(optim.Adam, betas=(mom,alpha), eps=eps)
    elif opt=='radam':       opt_func = partial(RAdam, betas=(mom,alpha), eps=eps)
    elif opt=='novograd':    opt_func = partial(Novograd, betas=(mom,alpha), eps=eps)
    elif opt=='rms':         opt_func = partial(optim.RMSprop, alpha=alpha, eps=eps)
    elif opt=='sgd':         opt_func = partial(optim.SGD, momentum=mom)
    elif opt=='ranger':      opt_func = partial(Ranger,  betas=(mom,alpha), eps=eps)
    elif opt=='ralamb':      opt_func = partial(Ralamb,  betas=(mom,alpha), eps=eps)
    elif opt=='rangerlars':  opt_func = partial(RangerLars,  betas=(mom,alpha), eps=eps)
    elif opt=='lookahead':   opt_func = partial(LookaheadAdam, betas=(mom,alpha), eps=eps)
    elif opt=='lamb':        opt_func = partial(Lamb, betas=(mom,alpha), eps=eps)
    if opt: learn.opt_func = opt_func

    # distributed
    if (gpu is not None) & (num_distrib()>1): learn.to_distributed(gpu)
    
    # to_fp16 
    learn.to_fp16()
    
    # train
    if not gpu: print(f"Starting training with max_lr: {max_lr}")
    if imagenet_pretrained:
        if not gpu: print("Training with transfer learning")
        # stage-1
        learn.freeze_to(-1)
        learn.fit_one_cycle(epochs, max_lr, callbacks=cbs)

        # stage-2
        lrs = slice(max_lr/100,max_lr/4)
        learn.freeze_to(-2)
        learn.fit_one_cycle(epochs, lrs, pct_start=0.8, callbacks=cbs)
 
        # stage-3
        lrs = slice(max_lr/100,max_lr/4)
        learn.unfreeze()
        learn.fit_one_cycle(epochs, lrs, pct_start=0.8, callbacks=cbs)
    else:
        if not gpu: print("Training from scratch")
        learn.fit_one_cycle(epochs, max_lr, callbacks=cbs)
        
    # modelexports
    if not nan_cb.isnan:
        if TEST: dtypes = ["Valid", "Test"]
        else: dtypes = ["Valid"]
        for dtype in dtypes:
            if not gpu: print(f"Generating Raw Predictions for {dtype}...")
            ds_type = getattr(DatasetType, dtype)
            preds, targs = learn.get_preds(ds_type)
            ds = learn.data.test_ds if dtype == "Test" else learn.data.valid_ds
            fnames = list(ds.items)
            try_save({"fnames":fnames, "preds":to_cpu(preds), "targs":to_cpu(targs)},
                     path=Path(EXPORT_PATH), file=f"{dtype}_raw_preds.pkl")
            if not gpu: print(f"Done.")
    else:
        if not gpu: print(f"Skipping Predictions due to NaN.")


### export

In [2]:
from local.notebook.export import *
# xport script
cells = read_nb("segmentation_training.ipynb")['cells']
src = cells[2]['source']
with open("segmentation_training.py", "w") as f: f.write(src)

###  `segmentation_training.py`

In [10]:
import fastai
from local.script import run_command

In [ ]:
rc = run_command(f"""
python segmentation_training.py \
--PATH=/home/turgutluk/.fastai/data/camvid \
--IMAGES=images \
--MASKS=labels \
--CODES=codes.txt \
--TRAIN=train.txt \
--VALID=0.2 \
--TEST=test.txt \
--bs=4 \
--size=(112,112) \
--imagenet_pretrained=1 \
--modelname=deeplabv3res50 \
--modelconfig="dict()" \
--tracking_metric=foreground_acc \
--void_name=Void \
--loss_function=cross_entropy \
--opt=radam \
--max_lr=3e-3 \
--epochs=10 \
--EXPORT_PATH=./experiment_export
""", logfn="./experiment_export/stdouterr.log")

To use this log_lamb_rs, please run 'pip install tensorboardx'. Also you must have Tensorboard running to see results
Print args here:
Training with model: deeplabv3res50 and config: {'pretrained': True, 'variant': 'D', 'skip': 'm1', 'skip_num': 48}
Training with metric: <function foreground_acc at 0x7f4d6ef917b8>
Training with loss: FlattenedLoss of CrossEntropyLoss()
Starting training with max_lr: 0.003
Training with transfer learning
epoch     train_loss  valid_loss  foreground_acc  time
Initializing self.best


In [ ]:
# rc = run_command(f"""
# python {Path(fastai.__file__).parent}/launch.py 
# --gpus=01 segmentation_training.py \
# --PATH=/home/turgutluk/.fastai/data/camvid \
# --IMAGES=images \
# --MASKS=labels \
# --CODES=codes.txt \
# --TRAIN=train.txt \
# --VALID=0.2 \
# --TEST=test.txt \
# --bs=4 \
# --size=112 \
# --imagenet_pretrained=1 \
# --modelname=resdunet18 \
# --modelconfig="dict()" \
# --tracking_metric="dice" \
# --void_name=None \
# --loss_function="cross_entropy" \
# --opt=radam \
# --max_lr=3e-3 \
# --epochs=10 \
# --EXPORT_PATH="."
# """, logfn="./experiment_export/stdouterr.log")

In [13]:
EXPORT_PATH = Path("./experiment_export/")

In [14]:
logs = json.loads(open(EXPORT_PATH/"stdouterr.log").read())

In [18]:
time, cmd, stderr, stdout =  logs[-1].values()

In [19]:
time

'10/01/2019, 22:45:24'

In [21]:
EXPORT_PATH.ls()

[PosixPath('experiment_export/training_log.csv'),
 PosixPath('experiment_export/models'),
 PosixPath('experiment_export/stdouterr.log')]

In [22]:
try:
    valid_preds = torch.load(EXPORT_PATH/'Valid_raw_preds.pkl')
    valid_preds['targs'][0]
except:pass

In [23]:
shutil.rmtree(EXPORT_PATH)

### fin